In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Created a list to append all tweet attributes(data)
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:louis030195').get_items()):
    if i>100:
        break
    attributes_container.append([tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])
    
# Creating a dataframe from the tweets list above 
tweets_df = pd.DataFrame(attributes_container, columns=["Date Created", "Number of Likes", "Source of Tweet", "Tweets"])
tweets_df

In [30]:
import os
import openai
import time
import re
values = open(".env", "r").read()
openai_api_key = re.findall(r"OPENAI_API_KEY=\"(.*)\"", values)[0]
openai_org = re.findall(r"OPENAI_ORG=\"(.*)\"", values)[0]
openai.api_key = openai_api_key
openai.organization = openai_org
def build_prompt(tweet):
  return f"""Tweet content: A rational person can find peace by cultivating indifference to things outside of their control.

Short list of main topics: rationality,wisdom,stoicism

###

Tweet content: Good product design requires an obsessed artist - it’s not a part-time job.

Short list of main topics: product-design,art,product

###

Tweet content: {tweet}

Short list of main topics:"""

delay = 5
for k, v in tweets_df.iterrows():
    tweet = v["Tweets"]
    prompt = build_prompt(tweet)
    print(f"Prompt: {prompt}")
    try:
        response = openai.Completion.create(
            model="code-davinci-002",
            prompt=prompt,
            temperature=0,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["###"]
        )
    # in case of RateLimitError, wait 1 minute and try again
    except openai.error.RateLimitError:
        time.sleep(60)
        continue
    if not response['choices'][0]['text']:
        print("Could not generate topics for tweet: ", tweet)
        time.sleep(delay)
        continue
    print(f"Completion: {response['choices'][0]['text']}")
    time.sleep(delay)
    from IPython. display import clear_output
    clear_output(wait=True)

Prompt: Tweet content: A rational person can find peace by cultivating indifference to things outside of their control.

Short list of main topics: rationality,wisdom,stoicism

###

Tweet content: Good product design requires an obsessed artist - it’s not a part-time job.

Short list of main topics: product-design,art,product

###

Tweet content: Your destiny is a result of your values,
Which are a result of your habits,
Which are a result of your actions,
Which are a result of your words,
Which are a result of your thoughts,
Which are a result of your beliefs.

Short list of main topics:
Completion:  destiny,values,habits




KeyboardInterrupt: 